## Redes Neurais Artificiais 2025.1

- **Disciplina**: Redes Neurais Artificiais 2025.1  
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  

## Equipe:

1. Afonso Henrique Torres Lucas | ifonso.developer@gmail.com | iFonso - 2215080047
2. Emanuelle Rocha Marreira | emanuellemarreira@gmail.com | emanuellemarreira - 2215080035
3. Erik Gustavo Lima de Oliveira | erik.exatas10@gmail.com | ErikExatas - 2115080049
4. David Augusto De Oliveira E Silva | david-augusto-silva - 2115080006
5. Ítalo Ferreira Fonseca
6. João Vitor Silva De Carvalho
7. Lilian Iazzai De Souza Oliveira | lilianiazzai@gmail.com | lilianiazzai - 2215080018
8. Vitor Nascimento Aguiar | Vtaguiar1909 - 2115080055

- **Github**: https://github.com/emanuellemarreira/image-authenticity-classification/tree/parte1

In [30]:
# bibliotecas
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, ConcatDataset
import numpy as np
import kagglehub
from collections import Counter
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, Subset
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

**Descrição da Atividade**

A tarefa de aprendizado de máquina definida neste trabalho é a classificação de imagens binária, com o objetivo de identificar se uma imagem é real (fotografia de pessoas reais) ou sintética, ou seja, gerada por Inteligência Artificial Utilizamos o dataset CIFAKE: Real and AI-Generated Synthetic Images, disponível no Kaggle, que contém imagens divididas em duas classes: "real" e "fake". A classe "real" é composta por fotografias autênticas de rostos humanos, enquanto a classe "fake" contém imagens de rostos geradas por algoritmos de IA, como GANs (Redes Generativas Adversariais). Como se trata de um problema visual e a tarefa envolve reconhecer padrões sutis na textura, iluminação e traços faciais que podem diferenciar uma imagem real de uma sintética, optamos por utilizar uma Rede Neural Convolucional (CNN), que é uma arquitetura especialmente eficaz para extração automática de características em tarefas de classificação de imagens.

Link do dataset: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images


### 1. Leitura do dataset

In [2]:
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

In [3]:
path

'/kaggle/input/cifake-real-and-ai-generated-synthetic-images'

In [4]:
#dataset_path = path
train_path = f"{path}/cifake/train"
test_path = f"{path}/cifake/test"

In [5]:
transform = transforms.Compose([
    # Resize da imagem pra 32x32 (pixels), seria bom colocar 128x128
    transforms.Resize((32, 32)),
    # PIL -> Tensor[canal, altura, largura] + Normalização de pixels
    transforms.ToTensor()
])

In [7]:
dataset = ImageFolder(root=path, transform=transform)

In [9]:
train_data = datasets.ImageFolder(root=f"{path}/train", transform=transform)
test_data = datasets.ImageFolder(root=f"{path}/test", transform=transform)

In [10]:
dataset

Dataset ImageFolder
    Number of datapoints: 120000
    Root location: /kaggle/input/cifake-real-and-ai-generated-synthetic-images
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

### 2. Mini Análise Exploratória

In [23]:
train_labels = train_data.targets

In [24]:
train_label_counts = Counter(train_labels)

In [25]:
train_label_counts # 0: FAKE / 1: REAL

Counter({0: 50000, 1: 50000})

In [14]:
train_data.classes

['FAKE', 'REAL']

### 3. Holdout estratificado: 70% treino / 30% validação

In [26]:
train_labels = train_data.targets

In [27]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_labels)), train_labels))

In [28]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [29]:
print(f"Treino: {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste: {len(test_data)} imagens")

Treino: 70000 imagens
Validação: 30000 imagens
Teste: 20000 imagens


### 4. K-Fold estratificado com k=5 sobre os 70% do treino

In [33]:
labels_train_split = np.array(train_labels)[train_idx]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [34]:
for fold, (k_train_idx, k_val_idx) in enumerate(kfold.split(np.zeros(len(train_split)), labels_train_split)):
    k_train = Subset(train_split, k_train_idx)
    k_val = Subset(train_split, k_val_idx)

    print(f"\nFold {fold+1}:")
    print(f" - Treino: {len(k_train)} imagens")
    print(f" - Validação: {len(k_val)} imagens")


Fold 1:
 - Treino: 56000 imagens
 - Validação: 14000 imagens

Fold 2:
 - Treino: 56000 imagens
 - Validação: 14000 imagens

Fold 3:
 - Treino: 56000 imagens
 - Validação: 14000 imagens

Fold 4:
 - Treino: 56000 imagens
 - Validação: 14000 imagens

Fold 5:
 - Treino: 56000 imagens
 - Validação: 14000 imagens
